# **Cantidad de docentes de IE públicas a nivel nacional, por área, edad, sexo, grado académico y modalidad**

Conjunto de datos estadísticos que presenta el número de docentes que laboran en instituciones educativas públicas del Perú durante el año lectivo 2024 (fuente: Censo Educativo del Ministerio de Educación del año 2024).

La información está agregada y se desglosa por:

- Área geográfica

    - Sexo (hombres, mujeres)

- Grupos de edad (25 años o menos; 26–35; 36–45; 46–55; 56–65; 66 años o más)

- Grado académico

- Modalidad / nivel educativo

Cada fila representa la combinación única de estas categorías con el total de docentes correspondiente. El dataset sirve para analizar la distribución del personal docente público y apoyar decisiones de política educativa, planeamiento y monitoreo del sector público

In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### **¿Qué haría primero si recibo tu dataset “Docentes IE públicas”?**

1. Verifico el contexto
- ¿De dónde viene el dataset? ¿Quién lo generó?
    - Censo Educativo MINEDU 2024
- ¿Cuál es el objetivo de negocio?
    - Analizar distribución de docentes por zona, brecha de género, brecha etaria, calidad profesional, brecha rural/urbana.
- ¿Para quién es este análisis?
    - Toma de decisiones de política educativa

2. Reviso la estructura del archivo



- ¿Qué formato tiene?

In [47]:
ruta = '../data/docentes_raw.csv'
df = pd.read_csv(ruta)

In [48]:
# Columnas
df.columns = df.columns.str.strip().str.replace(' ', '_').str.lower()
df.columns

Index(['cod_mod', 'anexo', 'codlocal', 'cen_edu', 'dir_cen', 'niv_mod',
       'nivel', 'nivel2', 'nivel3', 'ges_dep', 'gestion', 'codgeo', 'codooii',
       'area_censo', 'area', 'region', 'dpto', 'prov', 'dist', 'dre_ugel',
       'region_e', 'docentes_hombres', 'docentes_mujeres',
       'docentes_nombrados', 'docentes_contratados', 'docentes_25_menos_años',
       'docentes_2635_años', 'docentes_3645_años', 'docentes_4655_años',
       'docentes_5665_años', 'docentes_66_a_mas_años', 'docentes_con_titulo',
       'docentes_sin_titulo', 'docentes_total'],
      dtype='object')

- ¿Cuántas filas y columnas hay?

In [49]:
estructura = df.shape
print(f'Tiene {estructura[0]} filas y {estructura[1]} columnas.')

Tiene 66052 filas y 34 columnas.


- ¿Tiene claves únicas?

In [50]:
cod_mod_unique = len(df['cod_mod'].unique())
print(f'Contiene {cod_mod_unique} claves únicas en la columna "cod_mod"')

Contiene 66045 claves únicas en la columna "cod_mod"


- ¿Qué significan los campos?
    - Reviso el diccionario de datos y me aseguro de entender cada columna

3. Valido la calidad de los datos

- ¿Hay valores nulos?
    - No hay valores nulos

In [51]:
df.isnull().sum()[5:]

niv_mod                   0
nivel                     0
nivel2                    0
nivel3                    0
ges_dep                   0
gestion                   0
codgeo                    0
codooii                   0
area_censo                0
area                      0
region                    0
dpto                      0
prov                      0
dist                      0
dre_ugel                  0
region_e                  0
docentes_hombres          0
docentes_mujeres          0
docentes_nombrados        0
docentes_contratados      0
docentes_25_menos_años    0
docentes_2635_años        0
docentes_3645_años        0
docentes_4655_años        0
docentes_5665_años        0
docentes_66_a_mas_años    0
docentes_con_titulo       0
docentes_sin_titulo       0
docentes_total            0
dtype: int64

- ¿Hay duplicados?
    - No hay duplicados

In [52]:
df.duplicated().sum()

np.int64(0)

In [53]:
df = df.drop_duplicates(subset=['cod_mod'])

- ¿Los totales cuadran?

In [54]:
df.iloc[:,-3:]

,docentes_con_titulo,docentes_sin_titulo,docentes_total
0,6,0,6
1,4,0,4
2,2,0,2
3,3,0,3
4,2,0,2
...,...,...,...
66047,4,4,8
66048,4,4,8
66049,3,5,8
66050,0,5,5


- Filtrar y seleccionar las columnas de interés

In [55]:
# Filtrar niv_mod (B0-inicial, A2-primaria, FO-secundaria), ges_dep(A1-estatal)
df = df[(df['niv_mod'].isin(['B0', 'A2', 'F0'])) & (df['ges_dep'].isin(['A1']))]
# Seleccionar columnas de interés
df.drop(columns = ['gestion', 'codgeo', 'codooii', 'area', 'region', 'anexo', 'codlocal', 'cen_edu', 'nivel','nivel2', 'nivel3', 'ges_dep'], inplace = True)
df.head()

,cod_mod,dir_cen,niv_mod,area_censo,dpto,prov,dist,dre_ugel,region_e,docentes_hombres,...,docentes_contratados,docentes_25_menos_años,docentes_2635_años,docentes_3645_años,docentes_4655_años,docentes_5665_años,docentes_66_a_mas_años,docentes_con_titulo,docentes_sin_titulo,docentes_total
0,539213,EL MUYO S/N,A2,2,AMAZONAS,BAGUA,ARAMANGO,UGEL BAGUA,DRE AMAZONAS,0,...,0,1,2,2,1,0,0,6,0,6
1,585661,CALLE MIGUEL MONTEZA TAFUR S/N,A2,2,AMAZONAS,BAGUA,ARAMANGO,UGEL BAGUA,DRE AMAZONAS,0,...,1,1,1,0,1,1,0,4,0,4
2,708081,LA LIBERTAD S/N,A2,2,AMAZONAS,BAGUA,ARAMANGO,UGEL BAGUA,DRE AMAZONAS,0,...,0,0,0,1,1,0,0,2,0,2
3,708099,EL PORVENIR S/N,A2,2,AMAZONAS,BAGUA,ARAMANGO,UGEL BAGUA,DRE AMAZONAS,0,...,2,1,1,1,0,0,0,3,0,3
4,767939,AVENIDA DOS DE MAYO 557,A2,2,AMAZONAS,BAGUA,ARAMANGO,UGEL BAGUA,DRE AMAZONAS,0,...,0,0,0,0,2,0,0,2,0,2


- Tabla de dimensión niv_mod

In [56]:
dim_niv_mod = df[['niv_mod']].drop_duplicates()
dim_niv_mod['nivel_desc'] = dim_niv_mod['niv_mod'].map({
    'B0': 'Inicial',
    'A2': 'Primaria',
    'F0': 'Secundaria'
})
dim_niv_mod

,niv_mod,nivel_desc
0,A2,Primaria
31,B0,Inicial
86,F0,Secundaria


- Tabla de dimensión area_censo

In [57]:
dim_area = df[['area_censo']].drop_duplicates()
dim_area['area_desc'] = dim_area['area_censo'].map({
    1: 'Urbano',
    2: 'Rural'
})
dim_area

,area_censo,area_desc
0,2,Rural
98,1,Urbano


centro_educativo

In [58]:
centro_educativo = df[['cod_mod', 'dir_cen','niv_mod', 'area_censo', 'dpto', 'prov', 'dist', 'dre_ugel', 'region_e']]

docentes_hechos

In [59]:
docentes_hechos = df[['cod_mod', 'docentes_hombres', 'docentes_mujeres','docentes_nombrados', 'docentes_contratados', 'docentes_25_menos_años', 'docentes_2635_años',
                      'docentes_3645_años', 'docentes_4655_años', 'docentes_5665_años', 'docentes_66_a_mas_años', 'docentes_con_titulo', 'docentes_sin_titulo', 'docentes_total',]]

In [60]:
centro_educativo.to_csv('../data/centro_educativo.csv', index = False)
docentes_hechos.to_csv('../data/docentes_hechos.csv', index = False)
dim_niv_mod.to_csv('../data/dim_niv_mod.csv', index=False)
dim_area.to_csv('../data/dim_area.csv', index=False)